In [1]:
import bisect
import random
from typing import Any, Callable, Dict, Tuple, List
from collections import defaultdict

import numpy as np
import pandas as pd

from algorithms import A_star

In [2]:
# Wczytywanie danych

df = pd.read_csv("DaneSiec.csv", skiprows=3)
df

,<AS_nr>,<pref>,<1st_s>,<s_nr>,<ld_nr>,<del>,<jit>,<los>,<hop>
0,1,256,1,85,3356,39,47,0.0003,1
1,0,0,0,0,3705,23,10,0.0003,1
2,2,65536,86,85,34,21,7,0.0009,1
3,0,0,0,0,29076,49,37,0.0002,1
4,0,0,0,0,42226,33,46,0.0002,1
...,...,...,...,...,...,...,...,...,...
206969,393239,512,4191652,85,209,26,34,0.0008,1
206970,438802,5120,4191737,85,43802,10,49,0.0002,1
206971,12845938,256,4191822,85,12741,24,46,0.0004,1
206972,12845948,2048,4191907,85,20959,45,34,0.0001,1


In [3]:
# Problem powtórzeń między <AS_nr>, <ld_nr>

def data_transform(df: pd.DataFrame) -> Dict[int, Dict[int, float]]:
    data_transformed = {}
    df.drop((df[df["<ld_nr>"] == 0]).index, inplace=True)
    df.replace({"<AS_nr>": {0: np.nan}}, inplace=True)
    df["<AS_nr>"] = df["<AS_nr>"].ffill(axis=0).astype(int)
    df.drop_duplicates(inplace=True)
    for row in df.itertuples():
        start = row[1]
        end = row[5]
        dist = float(row[6])
        if (neighbours := data_transformed.get(start, None)) is not None:
            if end in neighbours:
                msg = f"Connection {start} -> {end} already exists"
                raise ValueError(msg)
            neighbours[end] = dist
        else:
            data_transformed[start] = {end: dist}
    return data_transformed

def get_neighbours(node: int, data: Dict[int, Dict[int, float]]) -> Dict[int, float]:
    return data[node]

def trivial_heurestic(*args, **kwargs) -> int:
    return 0.0

In [4]:
pathfinder = A_star(
    data_transform=data_transform,
    get_neighbours=get_neighbours,
    heurestic=trivial_heurestic,
    data=df
)

all_nodes = sorted(list(set(df["<AS_nr>"]).union(set(df["<ld_nr>"]))))

In [5]:
df

,<AS_nr>,<pref>,<1st_s>,<s_nr>,<ld_nr>,<del>,<jit>,<los>,<hop>
0,1,256,1,85,3356,39,47,0.0003,1
1,1,0,0,0,3705,23,10,0.0003,1
2,2,65536,86,85,34,21,7,0.0009,1
3,2,0,0,0,29076,49,37,0.0002,1
4,2,0,0,0,42226,33,46,0.0002,1
...,...,...,...,...,...,...,...,...,...
206969,393239,512,4191652,85,209,26,34,0.0008,1
206970,438802,5120,4191737,85,43802,10,49,0.0002,1
206971,12845938,256,4191822,85,12741,24,46,0.0004,1
206972,12845948,2048,4191907,85,20959,45,34,0.0001,1


In [6]:
x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
print(f"Finding path from {x1} to {x2}")

Finding path from 39688 to 46069


In [7]:
pathfinder(x1, x2)

{'path': [39688, 2854, 28917, 13122, 703, 2764, 46069], 'delay': 130.0}